# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.dropna()
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)

In [4]:
# Configure gmaps

Heatmap = gmaps.figure(center = [0,0] ,zoom_level = 2)
 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
Heatmap.add_layer(heat_layer)
Heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = cities_df.loc[(cities_df['Max Temp']<90) &
                           (cities_df['Max Temp']>60) &
                            (cities_df['Humidity']>45) &
                            (cities_df['Humidity']<70) &
                           (cities_df['Wind Speed']<20) &
                            (cities_df['Wind Speed']>5) &
                           (cities_df['Cloudiness']<50)].reset_index(drop=True)
narrowed_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
1,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05
2,68,beja,20,PT,1558378766,49,38.02,-7.87,71.60,14.99
3,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
4,78,crotone,20,IT,1558378768,67,39.09,17.12,62.60,10.29
5,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
6,93,antioch,40,US,1558378507,63,38.00,-121.81,64.99,10.29
7,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86
8,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
9,146,nardaran,40,AZ,1558378782,52,40.56,50.01,66.20,19.46


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:

hotel_df = narrowed_city_df.copy()
hotel_df['Hotel Name'] = ''
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41,
1,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05,
2,68,beja,20,PT,1558378766,49,38.02,-7.87,71.60,14.99,
3,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
4,78,crotone,20,IT,1558378768,67,39.09,17.12,62.60,10.29,
5,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
6,93,antioch,40,US,1558378507,63,38.00,-121.81,64.99,10.29,
7,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,
8,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
9,146,nardaran,40,AZ,1558378782,52,40.56,50.01,66.20,19.46,


In [7]:
params = {
    "radius" : 50000,
    "types": "Hotel",
    "keyword": "Lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    lat = row['Lat']
    lng = row['Lng']

    # update location key value
    params['location'] = f"{lat},{lng}"
    # make request
    response = requests.get(base_url, params=params).json()
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df = hotel_df.dropna()   
   
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,6,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41,Headlands Hotel Austinmer Beach
1,37,varhaug,0,NO,1558378761,63,58.61,5.65,64.99,8.05,Nodhagen turistforeningshytte
2,68,beja,20,PT,1558378766,49,38.02,-7.87,71.60,14.99,Herdade dos Grous
3,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Nascente do São Francisco
4,78,crotone,20,IT,1558378768,67,39.09,17.12,62.60,10.29,HOTEL ONDA BLEU
5,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Ecovilly
6,93,antioch,40,US,1558378507,63,38.00,-121.81,64.99,10.29,DoubleTree by Hilton Hotel & Spa Napa Valley -...
7,114,koumac,0,NC,1558378775,68,-20.56,164.28,74.40,12.86,Malabou Beach Hotel
8,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,Palissandre Cote Ouest resort & SPA
9,146,nardaran,40,AZ,1558378782,52,40.56,50.01,66.20,19.46,Hotel Fairmont Baku - Flame Towers


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
Heatmap.add_layer(markers)

# Display Map
Heatmap

Figure(layout=FigureLayout(height='420px'))